<a href="https://colab.research.google.com/github/eRuaro/Dog-Breed-Classifier/blob/main/dog_breed_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-end Dog Breed Classification with Tensorflow

This notebook builds a Dog Breed Classifier using Google Collab, Tensorflow, and Tensorflow Hub.

## 1. Problem

Identifying the breed of a dog given an image of a dog.

When I see a cute dog, I want to know what breed it is.

## 2. Data

The data is gathered from Kaggle's dog breed identification competition which houses over 20,000 dog breed images.

https://www.kaggle.com/c/dog-breed-identification/data

## 3. Evaluation

Testing is evaluated on Multi Class Log Loss between the predicted probability and the observed target.

For each image in the test set, predict a probability for each of the different breeds.

Example:
- What is the probability that this is a beagle?
- What is the probability that this is a labrador?

## 4. Features

Data information:
- Dealing with images (unstructured data)
- Due to image dataset, best to use deep learning / transfer learning
- There are 120 breeds of dogs in the dataset (120 different classes thus Multi Class Log Loss is used)
- Over 10,000 images each in both training, and test set.
- Test set images have no label as these are used for predicting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Setting up Our Data

In [2]:
!ls /content/drive/MyDrive/Tensorflow

dog_breed_classifier.ipynb  dog-breed-identification


In [3]:
#Unzipping a file in Colab
#!unzip "path-to-zip" -d "path-to-fodler-where-to-zip"

# Importing Tools

- Import Tensorflow ✅
- Import Tensorflow Hub ✅
- Use a GPU ✅

In [1]:
import tensorflow as tf
import tensorflow_hub as hub 

print("TF Version:", tf.__version__)
print("TF Hub Version:", hub.__version__)

print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

TF Version: 2.5.0
TF Hub Version: 0.12.0
GPU available
